# Test Task Sentences

Interactive notebook to test sentences and see model predictions with log probabilities.


In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer
from pathlib import Path
import sys

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent.parent))

from sparse_pretrain.src.pruning.run_pruning import load_model

# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model_path = "jacobcd52/ss_d128_f1"
tokenizer_name = "SimpleStories/SimpleStories-1.25M"

print(f"Loading model: {model_path}")
model, config = load_model(model_path, device=device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model loaded!")


Using device: cuda
Loading model: jacobcd52/ss_dense
Loading standalone model from HuggingFace Hub: jacobcd52/ss_dense
Model loaded!


In [2]:
config

{'model_config': {'n_layer': 4,
  'd_model': 512,
  'n_ctx': 512,
  'd_head': 16,
  'd_mlp': 2048,
  'vocab_size': 4096,
  'use_rms_norm': True,
  'tie_embeddings': False,
  'use_positional_embeddings': False,
  'use_bigram_table': False,
  'use_attention_sinks': True,
  'activation': 'gelu',
  'dropout': 0.0,
  'use_bias': True},
 'sparsity_config': {'enable_weight_sparsity': False,
  'target_l0_fraction': 1,
  'enable_activation_sparsity': False,
  'activation_topk_fraction': 0.25},
 'training_config': {'total_tokens': 2000000000,
  'batch_size': 128,
  'dataset_name': 'SimpleStories/SimpleStories',
  'tokenizer_name': 'SimpleStories/SimpleStories-1.25M'}}

In [3]:
def get_next_token_probs(context: str, top_k: int = 20):
    """
    Get top-k next token predictions with probabilities.
    
    Returns:
        List of (token_str, probability, log_prob) tuples
    """
    input_ids = tokenizer.encode(context, add_special_tokens=False, return_tensors="pt")
    input_ids = input_ids.to(device)
    
    with torch.no_grad():
        outputs = model(input_ids)
        if isinstance(outputs, tuple):
            logits = outputs[0]
        else:
            logits = outputs
        logits = logits[:, -1, :]  # (1, vocab_size)
        probs = F.softmax(logits, dim=-1)[0]
        log_probs = F.log_softmax(logits, dim=-1)[0]
    
    # Get top-k
    top_probs, top_indices = torch.topk(probs, top_k)
    
    results = []
    for i in range(top_k):
        token_id = top_indices[i].item()
        token_str = tokenizer.decode([token_id])
        prob = top_probs[i].item()
        log_prob = log_probs[token_id].item()
        results.append((token_str, prob, log_prob))
    
    return results, probs, log_probs


def show_predictions(context: str, top_k: int = 15):
    """Show top-k predictions for a context."""
    print(f"Context: \"{context}\"")
    print(f"\nTop {top_k} predictions:")
    print("-" * 50)
    
    results, probs, log_probs = get_next_token_probs(context, top_k)
    
    for i, (token_str, prob, log_prob) in enumerate(results):
        display_token = repr(token_str)
        print(f"{i+1:2}. {display_token:15} P={prob:.4f}  log P={log_prob:.2f}")
    
    return probs, log_probs


def compare_tokens(context: str, tokens: list):
    """Compare probabilities for multiple tokens."""
    print(f"Context: \"{context}\"")
    print(f"\nComparing tokens:")
    print("-" * 50)
    
    _, probs, log_probs = get_next_token_probs(context, top_k=1)
    
    for token in tokens:
        token_ids = tokenizer.encode(token, add_special_tokens=False)
        if len(token_ids) == 0:
            print(f"  {repr(token):15} NOT FOUND")
            continue
        
        token_id = token_ids[0]
        prob = probs[token_id].item()
        log_prob = log_probs[token_id].item()
        print(f"  {repr(token):15} P={prob:.4f}  log P={log_prob:.2f}")


In [12]:
# Test a sentence - edit this!
context = "when leo was hiding in the bush, mia approached"

show_predictions(context)


Context: "when leo was hiding in the bush, mia approached"

Top 15 predictions:
--------------------------------------------------
 1. 'him'           P=0.6026  log P=-0.51
 2. '.'             P=0.1171  log P=-2.14
 3. 'the'           P=0.0601  log P=-2.81
 4. 'leo'           P=0.0409  log P=-3.20
 5. 'slowly'        P=0.0317  log P=-3.45
 6. 'it'            P=0.0244  log P=-3.71
 7. 'with'          P=0.0149  log P=-4.21
 8. 'quietly'       P=0.0146  log P=-4.23
 9. 'her'           P=0.0130  log P=-4.34
10. ','             P=0.0129  log P=-4.35
11. 'them'          P=0.0127  log P=-4.37
12. 'and'           P=0.0113  log P=-4.48
13. 'carefully'     P=0.0102  log P=-4.58
14. 'cautious'      P=0.0060  log P=-5.12
15. 'his'           P=0.0036  log P=-5.63


(tensor([1.1741e-08, 2.1459e-06, 8.4641e-05,  ..., 3.5491e-09, 2.0298e-07,
         2.9445e-08], device='cuda:0'),
 tensor([-18.2602, -13.0520,  -9.3771,  ..., -19.4566, -15.4102, -17.3407],
        device='cuda:0'))

In [3]:
felt, rubbed, shook, raised


Context: "when Leo went to the park,"

Comparing tokens:
--------------------------------------------------
  ' he'           P=0.6685  log P=-0.40
  ' she'          P=0.0195  log P=-3.94
  ' they'         P=0.0488  log P=-3.02
  ' Leo'          P=0.0302  log P=-3.50
